In [94]:
import pandas as pd
import numpy as np
import cv2

In [101]:
annot = pd.read_csv("data/annotations.csv", sep='\t')
annot.head(5)

,attachment_id,text,user_id,height,width,length,train,begin,end
0,44e8d2a0-7e01-450b-90b0-beb7400d2c1e,Ё,185bd3a81d9d618518d10abebf0d17a8,1920,1080,156.0,True,36,112
1,df5b08f0-41d1-4572-889c-8b893e71069b,А,185bd3a81d9d618518d10abebf0d17a8,1920,1080,150.0,True,36,76
2,17f53df4-c467-4aff-9f48-20687b63d49a,Р,185bd3a81d9d618518d10abebf0d17a8,1920,1080,133.0,True,40,97
3,e3add916-c708-4339-ad98-7e2740be29e9,Е,185bd3a81d9d618518d10abebf0d17a8,1920,1080,144.0,True,43,107
4,bd7272ed-1850-48f1-a2a8-c8fed523dc37,Ч,185bd3a81d9d618518d10abebf0d17a8,1920,1080,96.0,True,20,70


In [96]:
annot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20400 entries, 0 to 20399
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   attachment_id  20400 non-null  object 
 1   text           20400 non-null  object 
 2   user_id        20400 non-null  object 
 3   height         20400 non-null  int64  
 4   width          20400 non-null  int64  
 5   length         20400 non-null  float64
 6   train          20400 non-null  bool   
 7   begin          20400 non-null  int64  
 8   end            20400 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 1.3+ MB


In [97]:
def crop_frame(frame):
    height, width = frame.shape[:2]
    th_dim = frame.shape[2]
    max_dim = max(height, width)
    dif = abs(height-width)

    first_side = dif // 2
    second_side = dif - first_side
    
    
    if width == max_dim:
        f_array = np.zeros(shape=(first_side, max_dim, th_dim))
        s_array = np.zeros(shape=(second_side, max_dim, th_dim))
        frame = np.concatenate((f_array, np.array(frame), s_array), axis=0)
    else:
        f_array = np.zeros(shape=(max_dim, first_side, th_dim))
        s_array = np.zeros(shape=(max_dim, second_side, th_dim))
        frame = np.concatenate((f_array, np.array(frame), s_array), axis=1)

    return frame

In [98]:
def load_video(path, img_size, i):
    cap = cv2.VideoCapture(path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        else:
            frame = crop_frame(frame)
            frame = cv2.resize(frame, (img_size, img_size))
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
            
    print(f"We are done on the image number {i}")
    return frames

In [99]:
for i, path in enumerate(annot['attachment_id'][:1]):
    full_path = "data/slovo/train/" + str(path) + ".mp4"
    annot['attachment_id'][i] = load_video(full_path, 300, i)

We are on the image number 0


C:\Users\84907\AppData\Local\Temp\ipykernel_29852\4213435768.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annot['attachment_id'][i] = load_video(full_path, 300, i)


In [100]:
annot['attachment_id'][0][0].shape

(300, 300, 3)